In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [2]:
pwd

'C:\\Users\\635718\\Desktop\\DataScience\\HackerEarth\\FutureDatathon'

In [3]:
sample = pd.read_csv('./Q2RecData/sampleSubmission.csv')

In [4]:
requiredCustomer = sample['customerID']

In [5]:
requiredCustomer.shape

(39205,)

In [6]:
print('BBID_2041' in requiredCustomer)

False


In [7]:
    
#df = pd.concat([chunk[chunk['customerID'] in requiredCustomer] for chunk in iter_csv])

In [8]:
# load product and sample file
products = pd.read_csv('./Q2RecData/products.csv', nrows=2000000)
sample = pd.read_csv('./Q2RecData/sampleSubmission.csv')

In [9]:
products.shape

(500000, 16)

In [10]:
sample.head(2)

,customerID,products
0,BBID_204221,"300663432,1000099534,1000475598,None,None,None..."
1,BBID_204254,"300663432,1000099534,1000475598,None,None,None..."


In [11]:
products.nunique()

customerID                    12564
DOB                            7770
Gender                            2
State                            40
PinCode                        1808
transactionDate                 789
store_code                        5
store_description                 5
till_no                          25
transaction_number_by_till    34465
promo_code                    19492
promotion_description         14426
product_code                  52459
product_description           52247
sale_price_after_promo        24027
discountUsed                      1
dtype: int64

In [12]:
products.head(2)

,customerID,DOB,Gender,State,PinCode,transactionDate,store_code,store_description,till_no,transaction_number_by_till,promo_code,promotion_description,product_code,product_description,sale_price_after_promo,discountUsed
0,BBID_2041,1981-10-18,male,NaN,452001.0,2016-01-19,2655,BB-INDORE-MALHAR MEGA MALL,2,51535,NONPROMO,NaN,300785147,BB-CB-16X20X168SWG-Suitable for ROI New,2.0,Payback
1,BBID_2041,1981-10-18,male,NaN,452001.0,2016-06-21,2655,BB-INDORE-MALHAR MEGA MALL,21,57836,NONPROMO,NaN,1000315939,SOO FRESH ROLL PIZZA 1PLT,40.0,Payback


In [13]:
#products[(products['customerID'] == 'BBID_2049382') | (products['customerID'] == 'BBID_2049678')]

In [14]:
products.groupby(by='customerID')['product_code'].count().sort_values(ascending=False).head()

customerID
BBID_204451     2308
BBID_2041210    1633
BBID_2041341    1429
BBID_2048499    1344
BBID_2043667    1303
Name: product_code, dtype: int64

In [9]:
#products[products['customerID'] == 'BBID_2049678']

In [10]:
misfit_customers = list(set(sample['customerID']) - set(products['customerID']))

In [11]:
39205 - len(misfit_customers)

8638

In [12]:
payment = pd.read_csv('./Q2RecData/tenderModes.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 866277 entries, 0 to 866276
Data columns (total 13 columns):
customerID                    866277 non-null object
DOB                           837289 non-null object
Gender                        774742 non-null object
State                         755746 non-null object
PinCode                       797283 non-null object
transactionDate               866277 non-null object
store_code                    866277 non-null int64
store_description             866277 non-null object
till_no                       866277 non-null int64
tender_type                   866277 non-null object
transaction_number_by_till    866277 non-null int64
payment_amount_by_tender      866277 non-null float64
PaymentUsed                   859212 non-null object
dtypes: float64(1), int64(3), object(9)
memory usage: 56.2+ MB


In [20]:
#Change the data to form clusters as we did in first part
#Then we predict the values for each cluster seperately

In [21]:
#Missing Data
total = products.isnull().sum().sort_values(ascending=False)
percent = (products.isnull().sum()/products.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
promotion_description,386743,0.773486
State,120727,0.241454
Gender,19737,0.039474
PinCode,778,0.001556
discountUsed,0,0.000000
sale_price_after_promo,0,0.000000


In [13]:
drop_columns_products = ['State','PinCode','store_description','till_no','promo_code','promotion_description',
                         'product_description','discountUsed','sale_price_after_promo']
products_cleaned = products.drop(drop_columns_products, axis=1)

In [23]:
#Payment Missing Data
total = payment.isnull().sum().sort_values(ascending=False)
percent = (payment.isnull().sum()/payment.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
State,110531,0.127593
Gender,91535,0.105665
PinCode,68994,0.079644
DOB,28988,0.033463
PaymentUsed,7065,0.008156
payment_amount_by_tender,0,0.000000


In [14]:
drop_columns_payment = ['DOB','Gender','State','PinCode','store_description','till_no','tender_type','PaymentUsed']
payment_cleaned = payment.drop(drop_columns_payment, axis=1)

In [25]:
payment_cleaned.columns

Index(['customerID', 'transactionDate', 'store_code',
       'transaction_number_by_till', 'payment_amount_by_tender'],
      dtype='object')

In [26]:
payment_cleaned.head()

,customerID,transactionDate,store_code,transaction_number_by_till,payment_amount_by_tender
0,BBID_2041,2016-01-19,2655,51535,672.00
1,BBID_2041,2016-04-07,2655,43394,35.00
2,BBID_2041,2016-05-01,2655,47586,34.20
3,BBID_2041,2016-03-19,2655,39335,92.00
4,BBID_2041,2016-03-20,2655,72624,32.75


In [27]:
products_cleaned.head(2)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code
0,BBID_2041,1981-10-18,male,2016-01-19,2655,51535,300785147
1,BBID_2041,1981-10-18,male,2016-06-21,2655,57836,1000315939


In [15]:
#grouping the data in the payment to combine the amount per transaction
payment_cleaned['totalSpend'] = payment_cleaned.groupby(['customerID','transactionDate','store_code',
                                                         'transaction_number_by_till']).transform('sum')

In [16]:
payment_cleaned.head()

,customerID,transactionDate,store_code,transaction_number_by_till,payment_amount_by_tender,totalSpend
0,BBID_2041,2016-01-19,2655,51535,672.00,672.00
1,BBID_2041,2016-04-07,2655,43394,35.00,35.00
2,BBID_2041,2016-05-01,2655,47586,34.20,34.20
3,BBID_2041,2016-03-19,2655,39335,92.00,92.00
4,BBID_2041,2016-03-20,2655,72624,32.75,32.75


In [17]:
payment_cleaned.drop('payment_amount_by_tender', axis=1, inplace=True)

In [18]:
payment_cleaned.drop_duplicates(inplace=True)

In [19]:
payment.shape

(866277, 13)

In [33]:
payment_cleaned.shape

(683778, 5)

In [20]:
products_cleaned.shape

(2000000, 7)

In [21]:
merge_on = ['customerID','transactionDate','store_code','transaction_number_by_till']
merged_data = pd.merge(products_cleaned,payment_cleaned,how='left',on=merge_on)

In [22]:
merged_data.shape

(2000000, 8)

In [37]:
merged_data.head(3)

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend
0,BBID_2041,1981-10-18,male,2016-01-19,2655,51535,300785147,672.0
1,BBID_2041,1981-10-18,male,2016-06-21,2655,57836,1000315939,40.0
2,BBID_2041,1981-10-18,male,2016-04-07,2655,43394,1000314595,35.0


In [38]:
merged_data['customerID'].value_counts().head(2)

BBID_204451     2308
BBID_2041210    1633
Name: customerID, dtype: int64

In [23]:
#add missing data
merged_data['totalSpend'].fillna(merged_data['totalSpend'].mean(), inplace=True)
merged_data['Gender'].fillna('male', inplace=True)
merged_data['DOB'].fillna(pd.to_datetime("1991-12-03"), inplace=True)

In [24]:
#Change Gender to categorical
merged_data['Gender'] = merged_data['Gender'].map( {'female': 0, 'male': 1} ).astype(int)

In [25]:
merged_data.loc[merged_data['DOB'] == 'NANA','DOB'] = pd.to_datetime("1991-12-03")

In [26]:
#get Age from DOB
merged_data['age'] = pd.to_datetime('today').year - pd.to_datetime(merged_data['DOB']).dt.year

In [27]:
#get day of purchase from transactionDate
merged_data['day_of_transaction'] = pd.to_datetime(merged_data['transactionDate']).dt.weekday

In [28]:
merged_data.head()

,customerID,DOB,Gender,transactionDate,store_code,transaction_number_by_till,product_code,totalSpend,age,day_of_transaction
0,BBID_2041,1981-10-18,1,2016-01-19,2655,51535,3.007851e+08,672.0,36,1
1,BBID_2041,1981-10-18,1,2016-06-21,2655,57836,1.000316e+09,40.0,36,1
2,BBID_2041,1981-10-18,1,2016-04-07,2655,43394,1.000315e+09,35.0,36,3
3,BBID_2041,1981-10-18,1,2016-03-19,2655,39335,3.009301e+08,92.0,36,5
4,BBID_2041,1981-10-18,1,2016-01-19,2655,51535,1.000010e+09,672.0,36,1


In [29]:
#Drop the final set of columns
merged_data.drop(['DOB','transactionDate','transaction_number_by_till'], axis=1, inplace=True)

In [41]:
merged_data.head()

,Gender,product_code,totalSpend,age,day_of_transaction
0,1,3.007851e+08,672.0,36,1
1,1,1.000316e+09,40.0,36,1
2,1,1.000315e+09,35.0,36,3
3,1,3.009301e+08,92.0,36,5
4,1,1.000010e+09,672.0,36,1


In [34]:
merged_data.nunique()

Gender                     2
product_code          116828
totalSpend             85374
age                       80
day_of_transaction         7
dtype: int64

In [40]:
total = merged_data.isnull().sum().sort_values(ascending=False)
percent = (merged_data.isnull().sum()/merged_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

,Total,Percent
day_of_transaction,0,0.0
age,0,0.0
totalSpend,0,0.0
product_code,0,0.0
Gender,0,0.0


In [39]:
#drop rows with product code as null
merged_data = merged_data[~pd.isnull(merged_data['product_code'])]

In [31]:
final_data = merged_data.drop(['Gender','store_code','totalSpend','age','day_of_transaction'], axis=1)

In [32]:
final_data.shape

(2000000, 2)

In [50]:
final_data.head()

,customerID,product_code
0,BBID_2041,300785147
1,BBID_2041,1000315939
2,BBID_2041,1000314595
3,BBID_2041,300930107
4,BBID_2041,1000010393


In [42]:
#Creating the clusters
#merged_data.drop(['customerID','store_code'], axis=1, inplace=True)

rbs = RobustScaler()
merger_data_transformed = rbs.fit_transform(merged_data)
#print(cld2[0])
    
km1 = KMeans(n_clusters=4)
km2 = km1.fit(merger_data_transformed)
label = km2.fit_predict(merger_data_transformed)

MemoryError: 

In [ ]:
#s_score = silhouette_samples(merger_data_transformed, label, sample_size=50000)
#print(s_score)

In [ ]:
final_data['label'] = label

In [ ]:
final_data[final_data['label'] == 3]

In [ ]:
final_data.info()

In [ ]:
final_data.groupby(by='label')['customerID'].count().sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
final_data.iloc[0][1]

In [ ]:
sns.distplot(final_data['label'])

In [ ]:
sns.jointplot(x='product_code',y='label',data=final_data)

In [ ]:
plt.scatter(final_data['product_code'],final_data['label'])

In [ ]:
#Create product recommendations for each cluster of users

In [ ]:
## how many recommendation to make
def take_top_(x):
    if x >= 20:
        return 1
    else:
        if x < 20:
            return int(np.round(20/x))

In [ ]:
## recommendation function
def recommend_affinity(user,product_summary,product_cooccurence_matrix):
    
    products_ = product_summary[product_summary['customerID'] == user]['product_collection'].iloc[0] # returns list
    take_top_ = product_summary[product_summary['customerID'] == user]['take_top'].iloc[0]
    
    recs = []
    
    if take_top_ == 0:
        return recs
    
    ## here we get the list of products which were bought the maximum number of times along with a particular productr
    for tt in products_:
        s = np.squeeze(np.asarray(product_cooccurence_matrix[tt].todense())) ## list of products bought with counts` with that product
        ll = s.argsort()[-int(take_top_):][::-1]
        recs.append(list(ll))

    recs = list(np.vstack(recs).flatten('F'))
 
    if not recs:
        return recs
    
    return recs

In [ ]:
import scipy.sparse as sps
def getSubmission(products_2):

    ## create product list by customers
    products_2 = products_2.groupby('customerID')['product_code'].apply(lambda x: x.tolist()).reset_index()
    ## remove duplicate products
    products_2['product_code'] = products_2['product_code'].map(lambda x: list(set(x)))
    ## fix product max len to 20 (we'll pick the last 20 i.e most recent ones)
    products_2['product_code'] = products_2['product_code'].map(lambda x: x[-20:])
    
    

    customerIDs = []
    product_codes = []

    for index, row in products_2.iterrows():
        #if index % 10 == 0:
        #    print (index)
        ls_len = len(row['product_code'])
        #print(ls_len)
        customerIDs.extend(np.repeat(row['customerID'], ls_len))
        product_codes.extend(row['product_code'])

    #encode values
    from sklearn.preprocessing import LabelEncoder
    lbl = LabelEncoder() ## for customers
    customerIDs = lbl.fit_transform(customerIDs)
    lbl2 = LabelEncoder() ## for products
    product_codes = lbl2.fit_transform(product_codes)
    
    n_unique_users = len(set(customerIDs))
    n_unique_products = len(set(product_codes))

    row = customerIDs
    col = product_codes

    vals = np.repeat(1, len(row))
    ## this matrix has information about a user bought which all products.
    user_product_matrix = sps.csr_matrix((vals, (row, col)), shape=(n_unique_users, n_unique_products))
    ## this matrix has information a product got bought along with which other products, and how many times
    product_cooccurence_matrix = (user_product_matrix.T * user_product_matrix)
    ## set diagonal equals to zero since we are not interested in knowing the count of a product with itself
    product_cooccurence_matrix.setdiag(np.repeat(0, product_cooccurence_matrix.shape[0]))

    ## create a data frame of encoded values
    product_summary = pd.DataFrame({'customerID':customerIDs, 'product_code':product_codes})
    product_summary = product_summary.groupby('customerID')['product_code'].agg(lambda x:x.tolist()).reset_index().rename(columns = {0:'product_collection'})
    
    def take_top_(x):
        if x >= 20:
            return 1
        else:
            if x < 20:
                return int(np.round(20/x))
    
    ## get count of products per customer
    product_summary['len_collection'] = product_summary['product_collection'].map(len)
    ## if a customer has 20 products in the list, we'll take 1 top most product per each product
    product_summary['take_top'] = product_summary['len_collection'].map(lambda x: take_top_(x))
    
    ## recommendation for customers
    unique_customers = []
    for i in sample['customerID']:
        if i not in misfit_customers:
            unique_customers.append(i)
    unique_customers = lbl.fit_transform(unique_customers)

    from collections import defaultdict
    out_dict = defaultdict(list)
    nulls = []
    a=0
    for user in unique_customers: #[:50]): ## i took first 50 customers. 
        #print(user)
        a=a+1
        #rec = recommend_affinity(user,product_summary,product_cooccurence_matrix)
        #print(product_summary[product_summary['customerID'] == user])
        products_ = product_summary[product_summary['customerID'] == user]['product_collection'].iloc[0] # returns list
        take_top_ = product_summary[product_summary['customerID'] == user]['take_top'].iloc[0]

        recs = []
        if take_top_ == 0:
            return recs

        ## here we get the list of products which were bought the maximum number of times along with a particular productr
        for tt in products_:
            s = np.squeeze(np.asarray(product_cooccurence_matrix[tt].todense())) ## list of products bought with counts` with that product
            ll = s.argsort()[-int(take_top_):][::-1]
            recs.append(list(ll))

        rec = list(np.vstack(recs).flatten('F'))
        #print(rec)

        if not rec:
            nulls.append(user)
        out_dict[user] = rec

    ## get unique items per customer
    from collections import defaultdict
    #print(out_dict)

    out_dict_2 = defaultdict(list)
    for k,v in out_dict.items():
        out_dict_2[k] = list(set(v))

    ## convert key values to inverse customer codes
    print(a)
    print(lbl.inverse_transform(customerIDs))
    customer_codes = list(lbl.inverse_transform(customerIDs))
    customer_maps = dict(zip(list(customerIDs), list(customer_codes)))

    out_dict_3 = defaultdict(list)
    for k,v in out_dict_2.items():
        out_dict_3[customer_maps[k]] = out_dict_2[k]
        mis_dict = defaultdict(list)

    for i in misfit_customers:
        mis_dict[i] = list(np.repeat('None', 20))

    submission = pd.DataFrame(list(out_dict_3.items()), columns=['customerID','products'])

    product_codes_inv = list(lbl2.inverse_transform(product_codes))
    product_maps = dict(zip(list(product_codes), list(product_codes_inv)))

    submission['products'] = submission['products'].map(lambda x: [product_maps.get(s) for s in x])

    for index, row in submission.iterrows():
        max_len = 20
        if len(row['products']) < max_len:
            size = len(row['products'])
            diff_ = max_len - size
            a = row['products']
            a.extend(np.repeat('None', diff_))
            submission.loc[index, 'products'] = a
        else:
            submission.loc[index, 'products'] = row['products'][:20]

    submission['products'] = submission['products'].map(lambda x: ','.join(str(e) for e in x))
    
    return submission

In [ ]:
final_data.groupby(by='label')['customerID'].count().sort_values(ascending=False)

In [ ]:
temp = final_data[final_data['label'] == 2].drop(['label'],axis=1)

In [ ]:
temp.nunique()

In [ ]:
#final_data[final_data['customerID'] == 'BBID_2049678']

In [ ]:
sub1 = getSubmission(temp)

In [ ]:
sub1